In [1]:
import argparse
import ast
import pathlib
import sys

import numpy as np
import pandas as pd
import pyarrow.parquet as pq
import toml
import torch
from sklearn import preprocessing

sys.path.append("../..")
from MLP_utils.parameters import Parameters
from MLP_utils.utils import (
    Dataset_formatter,
    optimized_model_create,
    output_stats,
    parameter_set,
    results_output,
    test_optimized_model,
    un_nest,
)
from sklearn.metrics import (
    accuracy_score,
    auc,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
    roc_auc_score,
    roc_curve,
)

sys.path.append("../../..")

In [ ]:
# set up the argument parser
argparser = argparse.ArgumentParser(description="MLP binary classification testing")

argparser.add_argument(
    "--cell_type",
    type=str,
    default="cells",
    help="The type of data to be used.",
)   
argparser.add_argument(
    "--control_name",
    type=str,
    default="control",
    help="The name of the control condition.",
)
argparser.add_argument(
    "--treatment_name",
    type=str,
    default="treatment",
    help="The name of the treatment condition.",
)
argparser.add_argument(
    "--shuffle",
    type=str
    default="False",
    help="Whether to shuffle the data before training.",
)

# get the arguments
args = argparser.parse_args()

CELL_TYPE = args.cell_type
CONTROL_NAME = args.control_name
TREATMENT_NAME = args.treatment_name
SHUFFLE = ast.literal_eval(args.shuffle)
print(f"CELL_TYPE: {CELL_TYPE} CONTROL_NAME: {CONTROL_NAME} TREATMENT_NAME: {TREATMENT_NAME} SHUFFLE: {SHUFFLE}")

In [3]:
MODEL_NAME = CONTROL_NAME + "_vs_" + TREATMENT_NAME

In [4]:
ml_configs_file = pathlib.Path("../../MLP_utils/binary_config.toml").resolve(
    strict=True
)
ml_configs = toml.load(ml_configs_file)
params = Parameters()
mlp_params = parameter_set(params, ml_configs)

# overwrite mlp_params via command line arguments from papermill
mlp_params.CELL_TYPE = CELL_TYPE
mlp_params.MODEL_NAME = MODEL_NAME
mlp_params.CONTROL_NAME = CONTROL_NAME
mlp_params.TREATMENT_NAME = TREATMENT_NAME
mlp_params.MODEL_NAME = MODEL_NAME
mlp_params.SHUFFLE = SHUFFLE

In [5]:
# Import Data
# set data file path under pathlib path for multi-system use
file_path = pathlib.Path(
    f"../../../data/{mlp_params.CELL_TYPE}_preprocessed_sc_norm.parquet"
).resolve(strict=True)

df = pq.read_table(file_path).to_pandas()

In [6]:
def test_loop(df, output_name, title, mlp_params):
    # Code snippet for metadata extraction by Jenna Tomkinson
    df_metadata = list(df.columns[df.columns.str.startswith("Metadata")])

    # define which columns are data and which are descriptive
    df_descriptive = df[df_metadata]
    df_values = df.drop(columns=df_metadata)
    # Creating label encoder
    le = preprocessing.LabelEncoder()
    # Converting strings into numbers
    print(df_values["oneb_Metadata_Treatment_Dose_Inhibitor_Dose"].unique().tolist())
    lst_of_treatments = (
        df_values["oneb_Metadata_Treatment_Dose_Inhibitor_Dose"].unique().tolist()
    )

    df_values["oneb_Metadata_Treatment_Dose_Inhibitor_Dose"] = le.fit_transform(
        df_values["oneb_Metadata_Treatment_Dose_Inhibitor_Dose"]
    )
    print(df_values["oneb_Metadata_Treatment_Dose_Inhibitor_Dose"].unique().tolist())
    lst_of_coded_treatments = (
        df_values["oneb_Metadata_Treatment_Dose_Inhibitor_Dose"].unique().tolist()
    )
    # make a dictionary of the treatments and their corresponding codes to decode later
    dict_of_treatments = {}
    for i, j in zip(
        lst_of_coded_treatments,
        lst_of_treatments,
    ):
        dict_of_treatments[i] = j
    # split into X and Y where Y are the predictive column and x are the observable data
    df_values_X = df_values.drop(
        [
            "oneb_Metadata_Treatment_Dose_Inhibitor_Dose",
            "twob_Metadata_Treatment_Dose_Inhibitor_Dose",
            "threeb_Metadata_Treatment_Dose_Inhibitor_Dose",
            "fourb_Metadata_Treatment_Dose_Inhibitor_Dose",
        ],
        axis=1,
    )
    df_values_Y = df_values["oneb_Metadata_Treatment_Dose_Inhibitor_Dose"]
    test_data = Dataset_formatter(
        torch.FloatTensor(df_values_X.values), torch.FloatTensor(df_values_Y.values)
    )

    mlp_params.IN_FEATURES = df_values_X.shape[1]
    print("Number of in features: ", mlp_params.IN_FEATURES)
    if mlp_params.MODEL_TYPE == "Regression":
        mlp_params.OUT_FEATURES = 1
    else:
        mlp_params.OUT_FEATURES = len(
            df_values["oneb_Metadata_Treatment_Dose_Inhibitor_Dose"].unique()
        )

    print("Number of out features: ", mlp_params.OUT_FEATURES)

    if mlp_params.OUT_FEATURES > 2:
        mlp_params.MODEL_TYPE = "Multi_Class"
    elif mlp_params.OUT_FEATURES == 2:
        mlp_params.OUT_FEATURES = mlp_params.OUT_FEATURES - 1
        mlp_params.MODEL_TYPE = "Binary_Classification"
    elif mlp_params.OUT_FEATURES == 1:
        mlp_params.MODEL_TYPE = "Regression"
    else:
        pass
    # convert data class into a dataloader to be compatible with pytorch
    test_loader = torch.utils.data.DataLoader(
        dataset=test_data, batch_size=1, shuffle=mlp_params.SHUFFLE
    )
    model, _ = optimized_model_create(mlp_params, mlp_params.MODEL_NAME)
    # calling the testing function and outputting list values of tested model
    if mlp_params.MODEL_TYPE == "Multi_Class" or mlp_params.MODEL_TYPE == "Regression":
        y_pred_list = test_optimized_model(
            model,
            test_loader,
            mlp_params,
            model_name=mlp_params.MODEL_NAME,
            shuffle=mlp_params.SHUFFLE,
        )
    elif mlp_params.MODEL_TYPE == "Binary_Classification":
        y_pred_list, y_pred_prob_list = test_optimized_model(
            model,
            test_loader,
            mlp_params,
            model_name=mlp_params.MODEL_NAME,
            shuffle=mlp_params.SHUFFLE,
        )
    else:
        raise Exception("Model type must be specified for proper model testing")

    # un-nest list if nested i.e. length of input data does not match length of output data
    if len(y_pred_list) != len(df_values_Y):
        y_pred_list = un_nest(y_pred_list)
        y_pred_prob_list = un_nest(y_pred_prob_list)
    else:
        pass

    stats, recall, precision, f1, precision_, recall_, threshold_ = output_stats(
        y_pred_list,
        df_values_Y,
        mlp_params,
        y_pred_prob_list,
        test_name=f"{output_name}_all_testing",
        model_name=mlp_params.MODEL_NAME,
        title=title,
        shuffle=mlp_params.SHUFFLE,
    )
    return (
        stats,
        recall,
        precision,
        f1,
        precision_,
        recall_,
        threshold_,
        dict_of_treatments,
    )

In [7]:
print(df["oneb_Metadata_Treatment_Dose_Inhibitor_Dose"].unique().tolist())

['media_ctr_0_Media_ctr_0.0', 'DMSO_0.100_DMSO_1.0', 'DMSO_0.100_Z-VAD-FMK_100.0', 'DMSO_0.100_Z-VAD-FMK_30.0', 'DMSO_0.100_DMSO_0.025', 'Thapsigargin_1.000_DMSO_0.025', 'Thapsigargin_10.000_DMSO_0.025', 'Topotecan_5.000_DMSO_0.025', 'Topotecan_10.000_DMSO_0.025', 'Topotecan_20.000_DMSO_0.025', 'LPS_0.010_DMSO_0.025', 'LPS_0.100_DMSO_0.025', 'LPS_1.000_DMSO_0.025', 'LPS_10.000_DMSO_0.025', 'LPS_10.000_Disulfiram_0.1', 'LPS_10.000_Disulfiram_1.0', 'LPS_10.000_Disulfiram_2.5', 'LPS_Nigericin_100.000_1.0_DMSO_0.025', 'LPS_Nigericin_100.000_3.0_DMSO_0.025', 'LPS_Nigericin_100.000_10.0_DMSO_0.025', 'Disulfiram_0.100_DMSO_0.025', 'Disulfiram_1.000_DMSO_0.025', 'Disulfiram_2.500_DMSO_0.025', 'H2O2_100.000_DMSO_0.025', 'LPS_10.000_Z-VAD-FMK_100.0', 'LPS_100.000_DMSO_0.025', 'LPS_Nigericin_1.000_1.0_DMSO_0.025', 'LPS_Nigericin_1.000_3.0_DMSO_0.025', 'LPS_Nigericin_1.000_10.0_DMSO_0.025', 'LPS_Nigericin_1.000_10.0_Disulfiram_1.0', 'LPS_Nigericin_1.000_10.0_Z-VAD-FMK_100.0', 'H2O2_100.000_Disulfi

In [8]:
paired_treatment_list = [
    ["DMSO_0.100_DMSO_0.025", "LPS_100.000_DMSO_0.025"],
    ["DMSO_0.100_DMSO_0.025", "Thapsigargin_1.000_DMSO_0.025"],
    ["DMSO_0.100_DMSO_0.025", "Thapsigargin_10.000_DMSO_0.025"],
    ["DMSO_0.100_DMSO_0.025", "LPS_0.100_DMSO_0.025"],
    ["DMSO_0.100_DMSO_0.025", "LPS_1.000_DMSO_0.025"],
    ["DMSO_0.100_DMSO_0.025", "LPS_10.000_DMSO_0.025"],
    ["DMSO_0.100_DMSO_0.025", "LPS_100.000_DMSO_0.025"],
    ["DMSO_0.100_DMSO_0.025", "Flagellin_0.100_DMSO_0.025"],
    ["DMSO_0.100_DMSO_0.025", "Flagellin_1.000_DMSO_0.025"],
    ["DMSO_0.100_DMSO_0.025", "Flagellin_1.000_Disulfiram_1.0"],
    ["DMSO_0.100_DMSO_0.025", "LPS_Nigericin_100.000_1.0_DMSO_0.025"],
    ["DMSO_0.100_DMSO_0.025", "LPS_Nigericin_100.000_3.0_DMSO_0.025"],
    ["DMSO_0.100_DMSO_0.025", "LPS_Nigericin_100.000_10.0_DMSO_0.025"],
    ["DMSO_0.100_DMSO_0.025", "LPS_Nigericin_1.000_1.0_DMSO_0.025"],
    ["DMSO_0.100_DMSO_0.025", "LPS_Nigericin_1.000_3.0_DMSO_0.025"],
    ["DMSO_0.100_DMSO_0.025", "LPS_Nigericin_1.000_10.0_DMSO_0.025"],
    ["DMSO_0.100_DMSO_0.025", "H2O2_100.000_Z-VAD-FMK_100.0"],
    ["DMSO_0.100_DMSO_0.025", "H2O2_100.000_DMSO_0.025"],
    ["LPS_100.000_DMSO_0.025", "Thapsigargin_1.000_DMSO_0.025"],
    ["LPS_100.000_DMSO_0.025", "Thapsigargin_10.000_DMSO_0.025"],
    ["LPS_10.000_DMSO_0.025", "Thapsigargin_1.000_DMSO_0.025"],
    ["LPS_10.000_DMSO_0.025", "Thapsigargin_10.000_DMSO_0.025"],
    ["LPS_1.000_DMSO_0.025", "Thapsigargin_1.000_DMSO_0.025"],
    ["LPS_1.000_DMSO_0.025", "Thapsigargin_10.000_DMSO_0.025"],
    ["LPS_0.100_DMSO_0.025", "Thapsigargin_1.000_DMSO_0.025"],
    ["LPS_0.100_DMSO_0.025", "Thapsigargin_10.000_DMSO_0.025"],
    ["LPS_0.010_DMSO_0.025", "Thapsigargin_1.000_DMSO_0.025"],
    ["LPS_0.010_DMSO_0.025", "Thapsigargin_10.000_DMSO_0.025"],
]

In [9]:
# create a dataframe to store the model stats
model_stats_df = pd.DataFrame(
    columns=[
        "treatments_tested",
        "model",
        "group",
        "shuffled_data",
        "PR_Threshold",
        "Precision",
        "Recall",
    ]
)
model_stats_df

,treatments_tested,model,group,shuffled_data,PR_Threshold,Precision,Recall


In [10]:
for i in paired_treatment_list:
    # filter df to only include the two treatments to test
    test_df = df.query(
        f"oneb_Metadata_Treatment_Dose_Inhibitor_Dose == '{i[0]}' | oneb_Metadata_Treatment_Dose_Inhibitor_Dose == '{i[1]}'"
    )
    output_name = ("__").join(
        test_df["oneb_Metadata_Treatment_Dose_Inhibitor_Dose"].unique()
    )

    print(output_name)

    title = f'{output_name.split("__")[0].split("_")[0]} vs {("__").join(output_name.split("__")[1].split("_")[:2])}'
    print(title)
    (
        stats,
        recall,
        precision,
        f1,
        precision_,
        recall_,
        threshold_,
        dict_of_treatments,
    ) = test_loop(test_df, output_name, title, mlp_params)
    print(recall, precision, f1)

    threshold_ = np.append(threshold_, None)
    stats_df = pd.DataFrame(
        {
            "PR_Threshold": threshold_,
            "Precision": precision_,
            "Recall": recall_,
        }
    )

    stats_df["treatments_tested"] = "0 vs 1"
    # make it so that the second treatment is always the one that is being tested as the positive label
    stats_df["treatments_tested"] = stats_df["treatments_tested"].replace(
        "0 vs 1", f"{dict_of_treatments[0]} vs {dict_of_treatments[1]}"
    )
    stats_df["model"] = mlp_params.MODEL_NAME
    stats_df["group"] = "test"
    stats_df["shuffled_data"] = mlp_params.SHUFFLE
    stats_df
    model_stats_df = pd.concat([model_stats_df, stats_df], axis=0)

DMSO_0.100_DMSO_0.025__LPS_100.000_DMSO_0.025
DMSO vs LPS__100.000
['DMSO_0.100_DMSO_0.025', 'LPS_100.000_DMSO_0.025']
[0, 1]
Number of in features:  1251
Number of out features:  2
DMSO_0.100_DMSO_0.025_vs_LPS_100.000_DMSO_0.025_shuffle
../../trained_models/model_save_states/Binary_Classification/SHSY5Y
DMSO_0.100_DMSO_0.025_vs_LPS_100.000_DMSO_0.025_shuffle
              precision    recall  f1-score   support

           0       0.69      0.69      0.69     35643
           1       0.31      0.31      0.31     15987

    accuracy                           0.57     51630
   macro avg       0.50      0.50      0.50     51630
weighted avg       0.57      0.57      0.57     51630

0.3102520798148496 0.3090342679127726 0.3096419764647127
DMSO_0.100_DMSO_0.025__Thapsigargin_1.000_DMSO_0.025
DMSO vs Thapsigargin__1.000
['DMSO_0.100_DMSO_0.025', 'Thapsigargin_1.000_DMSO_0.025']
[0, 1]
Number of in features:  1251
Number of out features:  2
DMSO_0.100_DMSO_0.025_vs_LPS_100.000_DMSO_0.025_shu

/tmp/ipykernel_1813769/4170310430.py:44: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  model_stats_df = pd.concat([model_stats_df, stats_df], axis=0)


DMSO_0.100_DMSO_0.025__Thapsigargin_10.000_DMSO_0.025
DMSO vs Thapsigargin__10.000
['DMSO_0.100_DMSO_0.025', 'Thapsigargin_10.000_DMSO_0.025']
[0, 1]
Number of in features:  1251
Number of out features:  2
DMSO_0.100_DMSO_0.025_vs_LPS_100.000_DMSO_0.025_shuffle
../../trained_models/model_save_states/Binary_Classification/SHSY5Y
DMSO_0.100_DMSO_0.025_vs_LPS_100.000_DMSO_0.025_shuffle
              precision    recall  f1-score   support

           0       0.73      0.97      0.83     35643
           1       0.26      0.03      0.05     13212

    accuracy                           0.72     48855
   macro avg       0.50      0.50      0.44     48855
weighted avg       0.60      0.72      0.62     48855

0.02512867090523766 0.2637013502779984 0.045884873194665186


/tmp/ipykernel_1813769/4170310430.py:44: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  model_stats_df = pd.concat([model_stats_df, stats_df], axis=0)


DMSO_0.100_DMSO_0.025__LPS_0.100_DMSO_0.025
DMSO vs LPS__0.100
['DMSO_0.100_DMSO_0.025', 'LPS_0.100_DMSO_0.025']
[0, 1]
Number of in features:  1251
Number of out features:  2
DMSO_0.100_DMSO_0.025_vs_LPS_100.000_DMSO_0.025_shuffle
../../trained_models/model_save_states/Binary_Classification/SHSY5Y
DMSO_0.100_DMSO_0.025_vs_LPS_100.000_DMSO_0.025_shuffle
              precision    recall  f1-score   support

           0       0.67      0.97      0.79     35643
           1       0.33      0.03      0.06     17510

    accuracy                           0.66     53153
   macro avg       0.50      0.50      0.43     53153
weighted avg       0.56      0.66      0.55     53153

0.03409480296973158 0.3325905292479109 0.06184926184926185


...

/tmp/ipykernel_1813769/4170310430.py:44: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  model_stats_df = pd.concat([model_stats_df, stats_df], axis=0)


Number of in features:  1251
Number of out features:  2
DMSO_0.100_DMSO_0.025_vs_LPS_100.000_DMSO_0.025_shuffle
../../trained_models/model_save_states/Binary_Classification/SHSY5Y
DMSO_0.100_DMSO_0.025_vs_LPS_100.000_DMSO_0.025_shuffle
              precision    recall  f1-score   support

           0       0.54      0.97      0.69     15859
           1       0.49      0.03      0.06     13766

    accuracy                           0.53     29625
   macro avg       0.51      0.50      0.38     29625
weighted avg       0.51      0.53      0.40     29625

0.03174487868661921 0.48936170212765956 0.05962207517566001
Thapsigargin_10.000_DMSO_0.025__LPS_0.010_DMSO_0.025
Thapsigargin vs LPS__0.010
['Thapsigargin_10.000_DMSO_0.025', 'LPS_0.010_DMSO_0.025']
[1, 0]


/tmp/ipykernel_1813769/4170310430.py:44: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  model_stats_df = pd.concat([model_stats_df, stats_df], axis=0)


Number of in features:  1251
Number of out features:  2
DMSO_0.100_DMSO_0.025_vs_LPS_100.000_DMSO_0.025_shuffle
../../trained_models/model_save_states/Binary_Classification/SHSY5Y
DMSO_0.100_DMSO_0.025_vs_LPS_100.000_DMSO_0.025_shuffle
              precision    recall  f1-score   support

           0       0.55      0.97      0.70     15859
           1       0.48      0.03      0.05     13212

    accuracy                           0.54     29071
   macro avg       0.51      0.50      0.38     29071
weighted avg       0.51      0.54      0.41     29071

0.028458976687859523 0.4759493670886076 0.05370661334095129


/tmp/ipykernel_1813769/4170310430.py:44: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  model_stats_df = pd.concat([model_stats_df, stats_df], axis=0)


In [11]:
model_stats_df

,treatments_tested,model,group,shuffled_data,PR_Threshold,Precision,Recall
0,DMSO_0.100_DMSO_0.025 vs LPS_100.000_DMSO_0.025,DMSO_0.100_DMSO_0.025_vs_LPS_100.000_DMSO_0.025,test,True,0.0,0.309646,1.000000
1,DMSO_0.100_DMSO_0.025 vs LPS_100.000_DMSO_0.025,DMSO_0.100_DMSO_0.025_vs_LPS_100.000_DMSO_0.025,test,True,0.0,0.309655,0.999875
2,DMSO_0.100_DMSO_0.025 vs LPS_100.000_DMSO_0.025,DMSO_0.100_DMSO_0.025_vs_LPS_100.000_DMSO_0.025,test,True,0.0,0.309661,0.999875
3,DMSO_0.100_DMSO_0.025 vs LPS_100.000_DMSO_0.025,DMSO_0.100_DMSO_0.025_vs_LPS_100.000_DMSO_0.025,test,True,0.0,0.309647,0.999812
4,DMSO_0.100_DMSO_0.025 vs LPS_100.000_DMSO_0.025,DMSO_0.100_DMSO_0.025_vs_LPS_100.000_DMSO_0.025,test,True,0.0,0.309653,0.999812
...,...,...,...,...,...,...,...
28440,LPS_0.010_DMSO_0.025 vs Thapsigargin_10.000_DM...,DMSO_0.100_DMSO_0.025_vs_LPS_100.000_DMSO_0.025,test,True,0.827824,0.467227,0.021041
28441,LPS_0.010_DMSO_0.025 vs Thapsigargin_10.000_DM...,DMSO_0.100_DMSO_0.025_vs_LPS_100.000_DMSO_0.025,test,True,0.830851,0.468013,0.021041
28442,LPS_0.010_DMSO_0.025 vs Thapsigargin_10.000_DM...,DMSO_0.100_DMSO_0.025_vs_LPS_100.000_DMSO_0.025,test,True,0.831917,0.468803,0.021041
28443,LPS_0.010_DMSO_0.025 vs Thapsigargin_10.000_DM...,DMSO_0.100_DMSO_0.025_vs_LPS_100.000_DMSO_0.025,test,True,0.835869,0.469595,0.021041


In [12]:
# set path for the model training metrics
metrics_path = pathlib.Path(
    f"../../results/{mlp_params.MODEL_TYPE}/{mlp_params.MODEL_NAME}/{mlp_params.CELL_TYPE}"
)
metrics_path.mkdir(parents=True, exist_ok=True)
# check if the model training metrics file exists
metrics_file = pathlib.Path(f"{metrics_path}/testing_metrics.csv")
if metrics_file.exists():
    metrics_df = pd.read_csv(metrics_file)
    if len(metrics_df["shuffled_data"].unique()) > 1:
        pass
    elif metrics_df["shuffled_data"].unique() == mlp_params.SHUFFLE:
        pass
    else:
        metrics_df = pd.concat([metrics_df, model_stats_df], axis=0)
        metrics_df.to_csv(metrics_file, index=False)
else:
    model_stats_df.to_csv(metrics_file, index=False)